# Loading Necessary Libraries and Models

In [84]:
# !pip install PyPDF2
# !pip install nltk
# !pip install wordcloud
# !pip install PyMuPDF
# from spacy.cli import download
# download("en_core_web_sm")
# !pip install pycryptodome
# !pip install bertopic
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# !pip install keybert
# !pip install -U sentence-transformers

In [134]:
import PyPDF2
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import spacy
import re
import matplotlib.pyplot as plt
from collections import Counter
import fitz  # PyMuPDF
from wordcloud import WordCloud
from spacy import displacy
from nltk.tokenize import word_tokenize
from bertopic import BERTopic
import os
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
stop = set(stopwords.words("english"))
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.util import ngrams
nlp = spacy.load("en_core_web_sm")
import ast
from sentence_transformers import SentenceTransformer, util
Sbert = SentenceTransformer('all-MiniLM-L6-v2')
import io
from PIL import Image

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Extracting PMBOK

In [87]:
pdf_file_path = "/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/PMBOK6-2017.pdf"
pdf_document = fitz.open(pdf_file_path)
extracted_text = ''
for page_num in range(430,493): #395+35 -> 458+35
    page = pdf_document[page_num]
    page_text = page.get_text()
    extracted_text += page_text
pdf_document.close()
# extracted_text

## Cleaning PMBOK

In [88]:
def clean_pmbok(extracted_text):
    patterns_and_replacements = [
        (r'Not For Distribution, Sale or Reproduction\.', ''),
        (r'Part 1 - Guide', ''),
        (r'\d{3}', ''),
        (r'(?<![a-zA-Z])u(?!.)', ''),
        (r'u ', ''),
        (r'\bn\b',''),
        (r'ﬁ', 'fi'),
        (r'\(', ''),
        (r'\)', ''),
        (r',', ''),
        (r'`', '')
        #(r'\n','')
    ]
    for pattern, replacement in patterns_and_replacements:
        extracted_text = re.sub(pattern, replacement, extracted_text)

    return extracted_text

## Section Extraction and Analysis

In [89]:
def extract_table_of_contents(pdf_path, start_page, end_page):
    pdf_document = fitz.open(pdf_path)
    table_of_contents = ""
    for page_num in range(start_page - 1, end_page):
        page = pdf_document.load_page(page_num)
        table_of_contents += page.get_text()

    pdf_document.close()
    return table_of_contents

table_of_contents_text = extract_table_of_contents(pdf_file_path, 11, 12)
print(table_of_contents_text[50:1000])


4 Develop Team................................................................................................................336
9.4.1 Develop Team: Inputs .......................................................................................339
9.4.2 Develop Team: Tools and Techniques ..............................................................340
9.4.3 Develop Team: Outputs ....................................................................................343
9.5 Manage Team................................................................................................................345
9.5.1 Manage Team: Inputs .......................................................................................347
9.5.2 Manage Team: Tools and Techniques ..............................................................348
9.5.3 Manage Team: Outputs ....................................................................................350
9.6 Contro


### Extracting RISK MANAGEMENT only

In [90]:
lines = table_of_contents_text.split('\n')
start = lines.index('11. PROJECT RISK MANAGEMENT .............................................................................................395')
finish = lines.index('12. PROJECT PROCUREMENT MANAGEMENT ............................................................................459')
lines = lines[start:finish]
print(lines)

['11. PROJECT RISK MANAGEMENT .............................................................................................395', '11.1 Plan Risk Management ..............................................................................................401', '11.1.1 Plan Risk Management: Inputs ......................................................................402', '11.1.2 Plan Risk Management: Tools and Techniques .............................................404', '11.1.3 Plan Risk Management: Outputs ...................................................................405', 'Not For Distribution, Sale or Reproduction.', 'VIII ', 'Table of Contents', '11.2 Identify Risks ..............................................................................................................409', '11.2.1 Identify Risks: Inputs .....................................................................................411', '11.2.2 Identify Risks: Tools and Techniques .....................................

### Cleaning Table of Contents

In [91]:
table_of_contents_text_clean = ''.join(lines)
table_of_contents_text_clean = table_of_contents_text_clean.replace('Not For Distribution, Sale or Reproduction','')
table_of_contents_text_clean = table_of_contents_text_clean.replace('Table of Contents','')

### Get titles

In [92]:
title_pattern = r'\s+([A-Za-z\s:]+)'
titles = []
title_matches = re.findall(title_pattern, table_of_contents_text_clean)
titles.extend(title_matches)
print(titles)

['PROJECT RISK MANAGEMENT ', 'Plan Risk Management ', 'Plan Risk Management: Inputs ', 'Plan Risk Management: Tools and Techniques ', 'Plan Risk Management: Outputs ', 'Identify Risks ', 'Identify Risks: Inputs ', 'Identify Risks: Tools and Techniques ', 'Identify Risks: Outputs ', 'Perform Qualitative Risk Analysis ', 'Perform Qualitative Risk Analysis: Inputs ', 'Perform Qualitative Risk Analysis: Tools and Techniques ', 'Perform Qualitative Risk Analysis: Outputs ', 'Perform Quantitative Risk Analysis ', 'Perform Quantitative Risk Analysis: Inputs ', 'Perform Quantitative Risk Analysis: Tools and Techniques ', 'Perform Quantitative Risk Analysis: Outputs ', 'Plan Risk Responses ', 'Plan Risk Responses: Inputs ', 'Plan Risk Responses: Tools and Techniques ', 'Plan Risk Responses: Outputs ', 'Implement Risk Responses ', 'Implement Risk Responses: Inputs ', 'Implement Risk Responses: Tools and Techniques ', 'Implement Risk Responses: Outputs', 'Monitor Risks ', 'Monitor Risks: Inputs '

### Get Page Numbers

In [93]:
page_number_pattern = r'\d{3}'
page_numbers = []
page_num_matches = re.findall(page_number_pattern, table_of_contents_text_clean)
page_numbers.extend(page_num_matches)
print(page_numbers)

['395', '401', '402', '404', '405', '409', '411', '414', '417', '419', '421', '422', '427', '428', '430', '431', '436', '437', '439', '441', '447', '449', '450', '451', '451', '453', '455', '456', '457']


### Get Sections

In [94]:
section_pattern = r'^(\d+(\.\d{1,3})*)'
sections = []
for line in lines:
    section_match = re.search(section_pattern, line)
    if section_match:
        sections.append(section_match.group(1))
print(sections)

['11', '11.1', '11.1.1', '11.1.2', '11.1.3', '11.2', '11.2.1', '11.2.2', '11.2.3', '11.3', '11.3.1', '11.3.2', '11.3.3', '11.4', '11.4.1', '11.4.2', '11.4.3', '11.5', '11.5.1', '11.5.2', '11.5.3', '11.6', '11.6.1', '11.6.2', '11.6.3', '11.7', '11.7.1', '11.7.2', '11.7.3']


### Verifying Lengths for each list

In [95]:
print(f' Sections : {len(sections)}')
print(f' Titles : {len(titles)}')
print(f' Page Numbers : {len(page_numbers)}')


 Sections : 29
 Titles : 29
 Page Numbers : 29


### Creating a Dataframe

In [96]:
data = {"Title": titles, "Starting_Page_Number": page_numbers, "Section": sections}
df = pd.DataFrame(data)
df.sample(12)

,Title,Starting_Page_Number,Section
12,Perform Qualitative Risk Analysis: Outputs,427,11.3.3
23,Implement Risk Responses: Tools and Techniques,451,11.6.2
8,Identify Risks: Outputs,417,11.2.3
17,Plan Risk Responses,437,11.5
20,Plan Risk Responses: Outputs,447,11.5.3
18,Plan Risk Responses: Inputs,439,11.5.1
26,Monitor Risks: Inputs,455,11.7.1
1,Plan Risk Management,401,11.1
28,Monitor Risks: Outputs,457,11.7.3
25,Monitor Risks,453,11.7


In [97]:
df.head()

,Title,Starting_Page_Number,Section
0,PROJECT RISK MANAGEMENT,395,11
1,Plan Risk Management,401,11.1
2,Plan Risk Management: Inputs,402,11.1.1
3,Plan Risk Management: Tools and Techniques,404,11.1.2
4,Plan Risk Management: Outputs,405,11.1.3


### Adding 'Ending Page Number' Column

In [98]:
page_numbers = df['Starting_Page_Number'].tolist()
end_pages = page_numbers
end_pages.pop(0)
end_pages.append('459')
print(end_pages)
df['Ending_Page_Number'] = end_pages

['401', '402', '404', '405', '409', '411', '414', '417', '419', '421', '422', '427', '428', '430', '431', '436', '437', '439', '441', '447', '449', '450', '451', '451', '453', '455', '456', '457', '459']


In [99]:
df['Title'] = df['Title'].str.rstrip()

In [100]:
df['Starting_Page_Number'] = df['Starting_Page_Number'].astype('int')
df['Ending_Page_Number'] = df['Ending_Page_Number'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 29 non-null     object
 1   Starting_Page_Number  29 non-null     int64 
 2   Section               29 non-null     object
 3   Ending_Page_Number    29 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ KB


In [101]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number
0,PROJECT RISK MANAGEMENT,395,11,401
1,Plan Risk Management,401,11.1,402
2,Plan Risk Management: Inputs,402,11.1.1,404
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405
4,Plan Risk Management: Outputs,405,11.1.3,409


### Content Extraction and Cleaning For Each Section

In [102]:
def content_extraction_each_section(pdf_file,Starting_Page_Number,Ending_Page_Number):
    pdf_document = fitz.open(pdf_file)
    extracted_text = ""
    for page_num in range(Starting_Page_Number +35, Ending_Page_Number+35):
        page = pdf_document.load_page(page_num)
        extracted_text += page.get_text()
    clean_text = clean_pmbok(extracted_text)
    pdf_document.close()
    return clean_text

In [103]:
def content_extraction_for_df(df):
    df['Content'] = df.apply(lambda x: content_extraction_each_section(pdf_file_path,x['Starting_Page_Number'],x['Ending_Page_Number']),axis=1)
    return df

In [104]:
content_extraction_for_df(df)
df['Content'] = df['Content'].str.replace('\n','')
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


In [105]:
df.loc[2].Content

' • Project   charter11.1 Plan RiskManagementEnterprise/Organization4.1Develop ProjectCharter• Project charterProject documents• Stakeholder register• Enterprise environmental factors• Organizational process assetsProject ManagementPlanProject management plan• All componentsProjectManagementPlanProjectDocuments• Risk management planFigure 11-3. Plan Risk Management: Data Flow DiagramThe Plan Risk Management process should begin when a project is conceived and should be completed early in the project. It may be necessary to revisit this process later in the project life cycle for example at a major phase change or if the project scope changes significantly or if a subsequent review of risk management effectiveness determines that the Project Risk Management process requires modification.11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PROJECT CHARTERDescribed in Section 4.1.3.1. The project charter documents the high-level project description and boundaries high-level requirements and risks.

### Cleaning Extracted Content
Some Sections end at the same page as another Section begins so we're removing extra contents that doesnt belong to a specific Section [getting the content of each section only when we find the section and the title in the content]

*example : Section '11.1' Title : 'Plan Risk Management' Content : ' • Project   charter11.1 Plan Risk Management'* -> Cleaned Content : '11.1 Plan Risk Management'

In [106]:
def extract_substring(row):
    start_string = row['Section'] + ' ' + row['Title'].upper()
    start_index = row['Content'].find(start_string)
    if start_index != -1:
        return row['Content'][start_index:]
    else:
      pass

In [107]:
df['Extracted_Content'] = df.apply(extract_substring, axis=1)
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Extracted_Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,None
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


In [108]:
df['Extracted_Content'].loc[23] = '11.6.2.1 EXPERT JUDGMENT Described in Section 4.1.2.1. Expertise should be considered from individuals or groups with specialized knowledge to validate or modify risk responses if necessary, and decide how to implement them in the most efficient and effective manner. 11.6.2.2 INTERPERSONAL AND TEAM SKILLS Interpersonal and team skills that can be used for this process include but are not limited to influencing. Some risk response actions may be owned by people outside the immediate project team or who have other competing demands. The project manager or person responsible for facilitating the risk process may need to exercise influencing (see Section 9.5.2.1) to encourage nominated risk owners to take necessary action where required. 11.6.2.3 PROJECT MANAGEMENT INFORMATION SYSTEM (PMIS) Described in Section 4.3.2.2. Project management information systems can include schedule, resource, and cost software to ensure that agreed-upon risk response plans and their associated activities are integrated into the project alongside other project activities. 11.6.3 IMPLEMENT RISK RESPONSES: OUTPUTS 11.6.3.1 CHANGE REQUESTS Described in Section 4.3.3.4. Implementation of risk responses may result in a change request to the cost and schedule baselines or other components of the project management plan. Change requests are processed for review and disposition through the Perform Integrated Change Control process (Section 4.6).'
df['Extracted_Content'].loc[0] = df['Content'].loc[0]

In [109]:
df['Extracted_Content'].isna().sum()

0

In [110]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Extracted_Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


In [111]:
def verifying_with_all_contents(df):
    for i, row in df.iterrows():
        print(f'For the {i}-th Content')
        title = row['Section'] + ' ' + row['Title'].upper()
        start = row['Starting_Page_Number']
        end = row['Ending_Page_Number']
        print(f'- With the Title = {title}')
        print(f'Starting in the Page : {start} Ending in the Page : {end}')
        print('- Content : ')
        print(row['Extracted_Content'])
        print('----------------')

verifying_with_all_contents(df)

For the 0-th Content
- With the Title = 11 PROJECT RISK MANAGEMENT
Starting in the Page : 395 Ending in the Page : 401
- Content : 
11PROJECT RISK MANAGEMENTProject Risk Management includes the processes of conducting risk management planning identification analysis response planning response implementation and monitoring risk on a project. The objectives of project risk management are to increase the probability and/or impact of positive risks and to decrease the probability and/or impact of negative risks in order to optimize the chances of project success.The Project Risk Management processes are:11.1 Plan Risk Management—The process of defining how to conduct risk management activities for a project.11.2 Identify Risks—The process of identifying individual project risks as well as sources of overall project risk and documenting their characteristics.11.3 Perform Qualitative Risk Analysis—The process of prioritizing individual project risks for further analysis or action by assessin

In [112]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Extracted_Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


### Removing Figures

In [113]:
# @title Saving Contents in Text files to remove Figures Manually
# output_directory = '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS1'
# os.makedirs(output_directory, exist_ok=True)
# def saving_each_extracted_content_in_a_text_file(df):
#   for index, row in df.iterrows():
#     file_name = f"{row['Section']}_{row['Title'].upper()}_{row['Starting_Page_Number']}_{row['Ending_Page_Number']}.txt"
#     file_path = os.path.join(output_directory,file_name)
#     with open(file_path, 'w') as file:
#       file.write(row['Extracted_Content'])
# saving_each_extracted_content_in_a_text_file(df)
# df.drop('Content', axis=1, inplace = True)
# df.drop('Extracted_Content', axis=1, inplace=True)

In [114]:
# @title cleaning file names to get the content easily
# def remove_digits_from_filenames(directory_path):
#     if not os.path.exists(directory_path):
#         print(f"Directory not found: {directory_path}")
#         return
#     for filename in os.listdir(directory_path):
#         if filename.endswith('.txt'):
#             file_path = os.path.join(directory_path, filename)
#             clean_filename = re.sub(r'\d+\.\d+', '', filename)
#             clean_filename = re.sub(r'\.\d+', '', filename)
#             clean_filename = re.sub(r'_', '', filename)
#             clean_filename = re.sub(r'\d', '', filename)
#             new_file_path = os.path.join(directory_path, clean_filename)
#             os.rename(file_path, new_file_path)
# remove_digits_from_filenames(r'/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS1')

In [115]:
# @title getting Contents without figures from text files
df['Title'] = df['Title'].str.replace(':','')
def add_clean_content_to_dataframe(df, directory_path):
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            title = os.path.splitext(filename)[0]
            matching_row = df[df['Title'].str.lower()  == title.lower()]
            if not matching_row.empty:
                with open(file_path, 'r') as file:
                    content = file.read()
                df.loc[matching_row.index[0], 'Content'] = content
add_clean_content_to_dataframe(df, r'/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS')

In [116]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Extracted_Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


### Removing Unecessary Columns

In [117]:
df.drop(['Starting_Page_Number', 'Ending_Page_Number','Section'], axis=1 , inplace = True)
df.rename(columns={'Extracted_Content': 'Description'}, inplace=True)
df.drop('Content', axis=1, inplace=True)
df.rename(columns={'Title':'Concept'}, inplace=True)
df.head()

,Concept,Description
0,PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


In [118]:
def clean_text(text):
    # Apply the provided cleaning code
    text = re.sub(r'\be\.g\b', 'for example', text)
    text = re.sub(r',', '', text)
    text = ' '.join([word.lower() for word in text.split() if word.lower() not in stop]) # Lowercase + removing stop words
    return text

## PMBOK CLEAN DATAFRAME

In [119]:
df['Cleaned_Description'] = df['Description'].apply(clean_text)
df.head()

,Concept,Description,Cleaned_Description
0,PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...
1,Plan Risk Management,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 plan risk managementplan risk management ...
2,Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...
3,Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques1...
4,Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...


In [123]:
df.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_1.csv')

# Extracting PMI

In [124]:
def extract_pdf(file_path,from_page_number,to_page_number):
    extracted_data = ""
    pdf = PyPDF2.PdfReader(file_path)
    for i in range(from_page_number,to_page_number):
        page = pdf.pages[i]
        extracted_data += page.extract_text()
    return extracted_data

In [126]:
figures_list=extract_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",10,12)

In [127]:
print(figures_list)

©2009 Project Management Institute. Practice Standard for Project Risk Management  LIST OF FIGURES  
 Figure 1-1. Hierarchy of PMI Project Risk Management Resources ....................................................2 
 Figure 1-2. Critical Success Factors for Project Risk Management .....................................................6  Figure 3-1. Project Risk Management Process Flow Diagram ...........................................................17  Figure 4-1. Key Areas of Focus for the Plan Risk Management Process ...........................................23  Figure 5-1. Three Perspectives of Risk Identification .........................................................................27  Figure 5-2. Cause, Risk, and Effect ......................................................................................................29  Figure 6-1. Building Risk Analysis Credibility .....................................................................................32 
 Figure 6-2. T

In [128]:
def cleaning_list_of_figures_PMI(list_of_figures):
    # Define a list of patterns and substitutions
    patterns_substitutions = [
        (r'ﬁ\s\s', 'fi'),  # Replace 'fi' with 'fi'
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),  # Remove footer
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),  # Remove the copyright notice
        (r'\.{2,}', ' '),  # Remove points
        (r'[^A-Za-z0-9\s\,\\-]+', ''),  # Remove special characters (keep digits, punctuation, and spaces)
        (r'\s+', ' '),  # Remove extra spaces
        (r'\S*LIST OF FIGURES\S*', ''),  # Remove text and characters attached to it
        (r'\s[0-9]+\s', ' '), #Remove page numbers
        (r'Figure\s([0-9]+)\s*-\s*([0-9]+)', r'Figure \1-\2.'),

    ]

    # Apply all patterns and substitutions in sequence
    for pattern, substitution in patterns_substitutions:
        list_of_figures = re.sub(pattern, substitution, list_of_figures)
    return list_of_figures

In [129]:
clean_lof=cleaning_list_of_figures_PMI(figures_list)
clean_lof

'  Figure 1-1. Hierarchy of PMI Project Risk Management Resources Figure 1-2. Critical Success Factors for Project Risk Management Figure 3-1. Project Risk Management Process Flow Diagram Figure 4-1. Key Areas of Focus for the Plan Risk Management Process Figure 5-1. Three Perspectives of Risk Identification Figure 5-2. Cause, Risk, and Effect Figure 6-1. Building Risk Analysis Credibility Figure 6-2. The Perform Qualitative Risk Analysis Process Figure 7-1. Comparison of Qualitative and Quantitative Approaches Figure 7-2. Structure of a Quantitative Risk Analysis Figure 8-1. Critical Success Factors for Risk Response Planning Figure 8-2. The Steps Involved in Planning Risk Responses Figure 9-1. Schematic Representation of the Monitor and Control Risks Process xi'

In [130]:
def create_figures_list(text):
    # Define a regular expression pattern to match figures
    figure_pattern = re.compile(r'Figure\s')

    # Replace figure patterns with a delimiter
    text_with_delimiter = re.sub(figure_pattern, r'\\\\\g<0>', text)

    # Split the text based on the delimiter
    figures = text_with_delimiter.split('\\')

    # Remove empty strings from the list
    figures = [figure.strip() for figure in figures if figure.strip()]

    return figures

In [131]:
figures = create_figures_list(clean_lof)
figures

['Figure 1-1. Hierarchy of PMI Project Risk Management Resources',
 'Figure 1-2. Critical Success Factors for Project Risk Management',
 'Figure 3-1. Project Risk Management Process Flow Diagram',
 'Figure 4-1. Key Areas of Focus for the Plan Risk Management Process',
 'Figure 5-1. Three Perspectives of Risk Identification',
 'Figure 5-2. Cause, Risk, and Effect',
 'Figure 6-1. Building Risk Analysis Credibility',
 'Figure 6-2. The Perform Qualitative Risk Analysis Process',
 'Figure 7-1. Comparison of Qualitative and Quantitative Approaches',
 'Figure 7-2. Structure of a Quantitative Risk Analysis',
 'Figure 8-1. Critical Success Factors for Risk Response Planning',
 'Figure 8-2. The Steps Involved in Planning Risk Responses',
 'Figure 9-1. Schematic Representation of the Monitor and Control Risks Process xi']

In [132]:
def extract_images_from_pdf(file_path, figures):
    output_folder = "figures"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    pdf_file = fitz.open(file_path)
    i = 0
    j = 1
    for page_index in range(len(pdf_file)):
        page = pdf_file[page_index]
        image_list = page.get_images(full=True)
        if image_list:
            print(f"[+] Found a total of {len(image_list)} images in page {page_index}")

        for image_index, img in enumerate(image_list, start=1):
            xref = img[0]
            base_image = pdf_file.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            pil_image = Image.open(io.BytesIO(image_bytes))
            if i < len(figures):
                image_filename = os.path.join(output_folder, f"{figures[i]}.{image_ext}")
                i = i + 1
            else:
                image_filename = os.path.join(output_folder, f"Figure D{j}.{image_ext}")
                j = j + 1
            pil_image.save(image_filename)

            print(f"    Image {image_index} saved as {image_filename}")
    pdf_file.close()
    return

In [135]:
extract_images_from_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",figures)

[+] Found a total of 1 images in page 13
    Image 1 saved as figures/Figure 1-1. Hierarchy of PMI Project Risk Management Resources.png
[+] Found a total of 1 images in page 17
    Image 1 saved as figures/Figure 1-2. Critical Success Factors for Project Risk Management.png
[+] Found a total of 1 images in page 28
    Image 1 saved as figures/Figure 3-1. Project Risk Management Process Flow Diagram.png
[+] Found a total of 1 images in page 34
    Image 1 saved as figures/Figure 4-1. Key Areas of Focus for the Plan Risk Management Process.png
[+] Found a total of 1 images in page 38
    Image 1 saved as figures/Figure 5-1. Three Perspectives of Risk Identification.png
[+] Found a total of 1 images in page 40
    Image 1 saved as figures/Figure 5-2. Cause, Risk, and Effect.png
[+] Found a total of 1 images in page 43
    Image 1 saved as figures/Figure 6-1. Building Risk Analysis Credibility.png
[+] Found a total of 1 images in page 44
    Image 1 saved as figures/Figure 6-2. The Perfor

## Loading the PMI Corpus

In [136]:
extracted_data_PMI = extract_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",12,123)

In [140]:
print(extracted_data_PMI[98:3000])

INTRODUCTION 
 Project Management Institute (PMI) practice standards are guides to the use of a tool, technique, or process 
identiﬁ  ed in  A Guide to the Project Management Body of Knowledge   ( PMBOK   ®  Guide  – Fourth Edition) or 
other PMI standards. Practice standards are targeted at audiences who participate in the management of 
projects. This includes project managers, project personnel, contract personnel, supervisors, and other project stakeholders. 
 A PMI practice standard describes processes, activities, inputs, and outputs for a speciﬁ  c Knowledge Area. 
It provides information on what the signiﬁ  cant process, tool, or technique is, what it does, why it is signiﬁ  cant, 
when it should be performed or executed, and, if necessary for further clariﬁ  cation, who should perform the process. A practice standard does not prescribe how the process is to be implemented, leaving that subject for other forums such as handbooks, manuals, and courses. 
 This chapter includes th

### PMI CLEANING

In [141]:
def cleaning_pmi(pmi_corpus):
    patterns_substitutions = [
        (r'ﬁ\s\s', 'fi'),
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),
        (r'[^A-Za-z0-9\s\.,-]+', ''),
        (r'\s+', ' '),
        (r'This chapter includes(.*?Factors for Project Risk Management)',''),
        (r'\s+', ' '),
    ]
    for pattern, substitution in patterns_substitutions:
        pmi_corpus = re.sub(pattern, substitution, pmi_corpus)
    for figure in figures:
        pmi_corpus = re.sub(r'\s' + re.escape(figure) + r'\s', ' ', pmi_corpus)


    return pmi_corpus

In [143]:
clean_pmi = cleaning_pmi(extracted_data_PMI)
clean_pmi[98:2000]

'of a tool, technique, or process identified in A Guide to the Project Management Body of Knowledge PMBOK Guide Fourth Edition or other PMI standards. Practice standards are targeted at audiences who participate in the management of projects. This includes project managers, project personnel, contract personnel, supervisors, and other project stakeholders. A PMI practice standard describes processes, activities, inputs, and outputs for a specific Knowledge Area. It provides information on what the significant process, tool, or technique is, what it does, why it is significant, when it should be performed or executed, and, if necessary for further clarification, who should perform the process. A practice standard does not prescribe how the process is to be implemented, leaving that subject for other forums such as handbooks, manuals, and courses. CHAPTER 1 INTRODUCTION 1.1 Purpose of the Practice Standard for Project Risk Management The purpose of the Practice Standard for Project Risk 

In [144]:
def extract_subsubsection_titles(text):
    patterns_to_remove = [
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),
        ('–', ''),
        ('  ', ' '),
        ('“|”', ''),
        (r'Inﬂ uence','Influence'),
        (r'[()/]', ''),
    ]

    for pattern, replacement in patterns_to_remove:
        text = re.sub(pattern, replacement, text)

    pattern = re.compile(r'[0-9]\.[0-9]\.[0-9]\.[0-9]|[A-Z]\.[0-9]\.[0-9]\.[0-9]+')
    matching_lines = [line.strip() for line in text.split('\n') if pattern.search(line)]

    return matching_lines
subsubtitles = extract_subsubsection_titles(extracted_data_PMI)
print(subsubtitles)

['D.1.1.1 The Risk Management Plan Purpose and Typical Content', 'D.1.1.2 Templates Relevant to Risk Management Planning', 'D.1.1.3 Risk Statement Metalanguage', 'D.1.1.4 Risk Prioritization and Selection Guidelines', 'Examples of techniques are given in D.2.1.1 through D.2.1.15.', 'D.2.1.1 Assumptions and Constraints Analysis', 'D.2.1.2 Brainstorming', 'D.2.1.3 Cause and Effect Ishikawa Diagrams', 'D.2.1.4 Checklists', 'D.2.1.5 Delphi Technique', 'D.2.1.6 Document Review', 'D.2.1.7 FMEAFault Tree Analysis', 'D.2.1.8 Force Field Analysis', 'D.2.1.9 Industry Knowledge Base', 'D.2.1.10 Influence Diagrams', 'D.2.1.11 Interviews', 'D.2.1.12 Nominal Group Technique', 'D.2.1.13 Post-Project ReviewsLessons LearnedHistorical Information', 'D.2.1.14 Prompt Lists', 'prompt lists have been developed for use in risk identiﬁ cation, and some of these are presented below. These can then be used as a framework for other risk identiﬁ cation techniques such as brainstorming see D.2.1.2 or risk intervie

### Extract table of contents from PMI

In [146]:
table_of_contents_PMI=extract_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",4,9)
print(table_of_contents_PMI[98:2000])


 CHAPTER 1  -  INTRODUCTION  .................................................................................................. .1
 1.1 Purpose of the Practice Standard for Project Risk Management  .......................... 2
 1.2 Project Risk Management Deﬁ  nition  ...................................................................... 4
 1.3 Role of Project Risk Management in Project Management  ................................... 4
 1.4 Good Risk Management Practice  ............................................................................ 5 1.5 Critical Success Factors for Project Risk Management  ......................................... 6
 1.6 Conclusion  .............................................................................................................. .7
 CHAPTER 2  -  PRINCIPLES AND CONCEPTS ..............................................................................9
 2.1 Introduction  .........................................................................

### Cleaning the table of contents

In [147]:
def cleaning_table_of_contents_PMI(table_of_contents_PMI):
    patterns_substitutions = [
        (r'ﬁ\s\s', 'fi'),
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),
        (r'\.{2,}', ' '),
        (r'\s.\d\s', ' \ '),
        (r'60B.3', r' \\ B.3'),
        (r'[^A-Za-z0-9\s\.,\\]+', ''),
        (r'(?<=\s)([0-9]+)(?=\s)', r'\1 \\ '),
        (r'\s+', ' '),
        (r'\S*TABLE OF CONTENTS\S*', ''),
        (r'(?<!CHAPTER) (\d+) \\ ', '  '),
        (r'CHAPTER (\d+) \\ ', r'CHAPTER \1 '),

    ]
    for pattern, substitution in patterns_substitutions:
        table_of_contents_PMI = re.sub(pattern, substitution, table_of_contents_PMI)
    # Define the patterns to match
    #patterns_to_remove = [re.compile(r'^\d+\.\d+\.\d+'), re.compile(r'^[A-Z]\.\d+\.\d+')]
    return table_of_contents_PMI


In [148]:
clean_toc_PMI=cleaning_table_of_contents_PMI(table_of_contents_PMI)
clean_toc_PMI[98:2000]

'roject Risk Management Definition \\ 1.3 Role of Project Risk Management in Project Management \\ 1.4 Good Risk Management Practice \\ 1.5 Critical Success Factors for Project Risk Management \\ 1.6 Conclusion \\ CHAPTER 2 PRINCIPLES AND CONCEPTS \\ 2.1 Introduction \\ 2.2 Definition of Project Risk \\ 2.3 Individual Risks and Overall Project Risk \\ 2.4 Stakeholder Risk Attitudes \\ 2.5 Iterative Process \\ 2.6 Communication \\ 2.7 Responsibility for Project Risk Management \\ 2.8 Project Managers Role for Project Risk Management \\ CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEMENT PROCESSES \\ 3.1 Project Risk Management and Project Management \\ 3.2 Project Risk Management Processes \\ CHAPTER 4 PLAN RISK MANAGEMENT \\ 4.1 Purpose and Objectives of the Plan Risk Management Process \\ 4.2 Critical Success Factors for the Plan Risk Management Process \\ 4.2.1 Identify and Address Barriers to Successful Project Risk Management \\ 4.2.2 Involve Project Stakeholders in Project Risk Mana

### Segment PMI into chapters

In [149]:
def segment_book_into_chapters(book_text):
    # Define the pattern for chapter headings (assuming they start with "CHAPTER X - TITLE")
    pattern = re.compile(r'CHAPTER \d [A-Z ]+ |APPENDIX [A-Z ] ')
    chapter_headings = re.findall(pattern, book_text)
    chapter_segments = re.split(pattern, book_text)
    columns = ['Chapter', 'Content']
    df = pd.DataFrame(columns=columns)

    for heading, content in zip(chapter_headings, chapter_segments[1:]):
        heading = heading.strip().upper()
        content = content.strip()
        if heading in df['Chapter'].values:
            idx = df.index[df['Chapter'] == heading].tolist()[0]
            df.at[idx, 'Content'] += ' ' + content
        else:
            df = df.append({'Chapter': heading, 'Content': content}, ignore_index=True)

    return df

In [150]:
#Segment PMI into chapters
chapters_data = segment_book_into_chapters(clean_pmi)
chapters_data

,Chapter,Content
0,CHAPTER 1 INTRODUCTION,Project Management Institute PMI practice stan...
1,CHAPTER 2 PRINCIPLES AND CONCEPTS,2.1 Introduction This chapter introduces the k...
2,CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEM...,3.1 Project Risk Management and Project Manage...
3,CHAPTER 4 PLAN RISK MANAGEMENT,4.1 Purpose and Objectives of the Plan Risk Ma...
4,CHAPTER 5 IDENTIFY RISKS,5.1 Purpose and Objectives of the Identify Ris...
5,CHAPTER 6 PERFORM QUALITATIVE RISK ANALYSIS,6.1 Purpose and Objectives of the Perform Qual...
6,CHAPTER 7 PERFORM QUANTITATIVE RISK ANALYSIS,7.1 Purpose and Objectives of the Perform Quan...
7,CHAPTER 8 PLAN RISK RESPONSES,The Plan Risk Responses process determines eff...
8,CHAPTER 9 MONITOR AND CONTROL RISKS,The effectiveness of Project Risk Management d...
9,APPENDIX A,GUIDELINES FOR A PMI PRACTICE STANDARD A.1 Int...


### Extract the chapters from the table of contents

In [151]:
def segment_toc_into_chapters(book_text):
    # Define the pattern for chapter headings (assuming they start with "CHAPTER X - TITLE")
    pattern = re.compile(r'CHAPTER \d [A-Z ]+ |APPENDIX [A-Z ] ')
    chapter_headings = re.findall(pattern, book_text)
    chapter_segments = re.split(pattern, book_text)
    chapters_data = {}
    for heading, content in zip(chapter_headings, chapter_segments[1:]):
        heading = heading.strip()
        content = content.strip()
        key = heading.upper()
        if key in chapters_data:
            chapters_data[key] += ' ' + content
        else:
            chapters_data[key] = content
    return chapters_data

In [153]:
toc_data = segment_toc_into_chapters(clean_toc_PMI)
toc_data

{'CHAPTER 1 INTRODUCTION': '\\ 1.1 Purpose of the Practice Standard for Project Risk Management \\ 1.2 Project Risk Management Definition \\ 1.3 Role of Project Risk Management in Project Management \\ 1.4 Good Risk Management Practice \\ 1.5 Critical Success Factors for Project Risk Management \\ 1.6 Conclusion \\',
 'CHAPTER 2 PRINCIPLES AND CONCEPTS': '\\ 2.1 Introduction \\ 2.2 Definition of Project Risk \\ 2.3 Individual Risks and Overall Project Risk \\ 2.4 Stakeholder Risk Attitudes \\ 2.5 Iterative Process \\ 2.6 Communication \\ 2.7 Responsibility for Project Risk Management \\ 2.8 Project Managers Role for Project Risk Management \\',
 'CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEMENT PROCESSES': '\\ 3.1 Project Risk Management and Project Management \\ 3.2 Project Risk Management Processes \\',
 'CHAPTER 4 PLAN RISK MANAGEMENT': '\\ 4.1 Purpose and Objectives of the Plan Risk Management Process \\ 4.2 Critical Success Factors for the Plan Risk Management Process \\ 4.2.1 Id

### Extract all the titles from the table of contents

In [154]:
def Extract_title_and_subtitles_toc(chapters_data):
    updated_chapters_data = {}

    for chapter, content in chapters_data.items():
        sections = content.split(' \\ ')
        sections = [section.strip() for section in sections if section.strip()]
        sections = [re.sub(r'\\', '', section) for section in sections if section.strip()]
        updated_chapters_data[chapter] = sections
    return updated_chapters_data

In [155]:
updated_toc_data = Extract_title_and_subtitles_toc(toc_data)
updated_toc_data

{'CHAPTER 1 INTRODUCTION': [' 1.1 Purpose of the Practice Standard for Project Risk Management',
  '1.2 Project Risk Management Definition',
  '1.3 Role of Project Risk Management in Project Management',
  '1.4 Good Risk Management Practice',
  '1.5 Critical Success Factors for Project Risk Management',
  '1.6 Conclusion '],
 'CHAPTER 2 PRINCIPLES AND CONCEPTS': [' 2.1 Introduction',
  '2.2 Definition of Project Risk',
  '2.3 Individual Risks and Overall Project Risk',
  '2.4 Stakeholder Risk Attitudes',
  '2.5 Iterative Process',
  '2.6 Communication',
  '2.7 Responsibility for Project Risk Management',
  '2.8 Project Managers Role for Project Risk Management '],
 'CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEMENT PROCESSES': [' 3.1 Project Risk Management and Project Management',
  '3.2 Project Risk Management Processes '],
 'CHAPTER 4 PLAN RISK MANAGEMENT': [' 4.1 Purpose and Objectives of the Plan Risk Management Process',
  '4.2 Critical Success Factors for the Plan Risk Managemen

### Extract only the section titles from the table of contents

In [156]:
section_pattern = re.compile(r'\s*([0-9]\.[0-9]|[A-Z]\.[0-9])\s')
chapter_sections = {}
for chapter, sections in updated_toc_data.items():
    section_titles = [section.strip() for section in sections if section_pattern.match(section)]
    chapter_sections[chapter] = section_titles
chapter_sections

{'CHAPTER 1 INTRODUCTION': ['1.1 Purpose of the Practice Standard for Project Risk Management',
  '1.2 Project Risk Management Definition',
  '1.3 Role of Project Risk Management in Project Management',
  '1.4 Good Risk Management Practice',
  '1.5 Critical Success Factors for Project Risk Management',
  '1.6 Conclusion'],
 'CHAPTER 2 PRINCIPLES AND CONCEPTS': ['2.1 Introduction',
  '2.2 Definition of Project Risk',
  '2.3 Individual Risks and Overall Project Risk',
  '2.4 Stakeholder Risk Attitudes',
  '2.5 Iterative Process',
  '2.6 Communication',
  '2.7 Responsibility for Project Risk Management',
  '2.8 Project Managers Role for Project Risk Management'],
 'CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEMENT PROCESSES': ['3.1 Project Risk Management and Project Management',
  '3.2 Project Risk Management Processes'],
 'CHAPTER 4 PLAN RISK MANAGEMENT': ['4.1 Purpose and Objectives of the Plan Risk Management Process',
  '4.2 Critical Success Factors for the Plan Risk Management Proce

### Segment each chapter into sections

In [157]:
def segment_chapter_into_sections(chapters_data):
    updated_chapters_data = {}
    for index, row in chapters_data.iterrows():
        chapter = row['Chapter']
        content = row['Content']
        updated_content = re.sub(r'\s([0-9]\.[0-9]|[A-Z]\.[0-9])\s', r' \\ \1 ', content)
        sections = updated_content.split(' \\ ')
        sections = [section.strip() for section in sections if section.strip()]
        updated_chapters_data[chapter] = sections
    data_for_df = [{'Chapter': chapter, 'Section': section} for chapter, sections in updated_chapters_data.items() for section in sections]
    df = pd.DataFrame(data_for_df)

    return df

In [158]:
updated_chapters_data = segment_chapter_into_sections(chapters_data)
updated_chapters_data

,Chapter,Section
0,CHAPTER 1 INTRODUCTION,Project Management Institute PMI practice stan...
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition The def...
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice Project Risk...
...,...,...
61,APPENDIX D,"D.4 Techniques, Examples and Templates for Per..."
62,APPENDIX D,8.0 Primavera Pertmaster Figure D15. Example o...
63,APPENDIX D,"D.5 Techniques, Examples, and Templates for Pl..."
64,APPENDIX D,"D.6 Techniques, Examples and Templates for Mon..."


### Create the Section_title column

In [159]:
updated_chapters_data['Section_title'] = None
for index, row in updated_chapters_data.iterrows():
    section_title = ''
    for chapter, section_titles in chapter_sections.items():
        for title in section_titles:
            if title.strip().lower() in row['Section'].strip().lower():
                section_title = title.strip()
                updated_chapters_data.at[index, 'Section'] = row['Section'].replace(title, '').strip()
                break
        if section_title:
            break
    updated_chapters_data.at[index, 'Section_title'] = section_title
updated_chapters_data = updated_chapters_data[['Chapter', 'Section_title', 'Section']]
updated_chapters_data

,Chapter,Section_title,Section
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...
...,...,...,...
61,APPENDIX D,,"D.4 Techniques, Examples and Templates for Per..."
62,APPENDIX D,,8.0 Primavera Pertmaster Figure D15. Example o...
63,APPENDIX D,"D.5 Techniques, Examples, and Templates for Pl...",8 Plan Risk Responses develops the set of acti...
64,APPENDIX D,,"D.6 Techniques, Examples and Templates for Mon..."


### Extract sub-titles from the table of contents

In [160]:
# Define a pattern to match section titles like '1.1', '2.2', etc.
subsection_pattern = re.compile(r'\s*([0-9]\.[0-9]\.[0-9]+|[A-Z]\.[0-9]\.[0-9]+)\s')
chapter_subsections = {}
for chapter, subsections in updated_toc_data.items():
    subsection_titles = [subsection.strip() for subsection in subsections if subsection_pattern.match(subsection)]
    chapter_subsections[chapter] = subsection_titles
chapter_subsections

{'CHAPTER 1 INTRODUCTION': [],
 'CHAPTER 2 PRINCIPLES AND CONCEPTS': [],
 'CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEMENT PROCESSES': [],
 'CHAPTER 4 PLAN RISK MANAGEMENT': ['4.2.1 Identify and Address Barriers to Successful Project Risk Management',
  '4.2.2 Involve Project Stakeholders in Project Risk Management',
  '4.2.3 Comply with the Organizations Objectives, Policies, and Practices',
  '4.3.1 Planning Sessions',
  '4.3.2 Templates'],
 'CHAPTER 5 IDENTIFY RISKS': ['5.2.1 Early Identification',
  '5.2.2 Iterative Identification',
  '5.2.3 Emergent Identification',
  '5.2.4 Comprehensive Identification',
  '5.2.5 Explicit Identification of Opportunities',
  '5.2.6 Multiple Perspectives',
  '5.2.7 Risks Linked to Project Objectives',
  '5.2.8 Complete Risk Statement',
  '5.2.9 Ownership and Level of Detail',
  '5.2.10 Objectivity',
  '5.3.1 Historical Review',
  '5.3.2 Current Assessments',
  '5.3.3 Creativity Techniques'],
 'CHAPTER 6 PERFORM QUALITATIVE RISK ANALYSIS': ['6.2.1

### Segment section into subsections

In [161]:
def segment_sections_into_subsections(updated_clean_pmi):
    data_for_df = []
    current_chapter = None
    current_section_title = None
    current_section = None
    for index, row in updated_clean_pmi.iterrows():
        chapter = row['Chapter']
        section_title = row['Section_title']
        section = row['Section']
        matches = list(re.finditer(r'\s([0-9]\.[0-9]\.[0-9]|[A-Z]\.[0-9]\.[0-9])\s', section))
        if matches:
            for match in matches:
                title_index = match.start()
                if title_index > 8 and (section[title_index - 8:title_index].lower() == 'sections' or section[title_index - 8:title_index].lower() == 'through'):
                    continue
                section_after_title = section[title_index:]
                updated_section = re.sub(r'\s([0-9]\.[0-9]\.[0-9]+|[A-Z]\.[0-9]\.[0-9]+)\s', r' \\ \1 ', section_after_title)
                subsections = updated_section.split(' \\ ')
                subsections = [subsection.strip() for subsection in subsections if subsection.strip()]
                if subsections:
                    current_chapter = chapter
                    current_section_title = section_title
                for subsection in subsections:
                    data_for_df.append({'Chapter': current_chapter, 'Section_title': current_section_title, 'Section': section, 'Subsection': subsection})
        else:
            data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section, 'Subsection': ''})
    df = pd.DataFrame(data_for_df)
    return df

In [162]:
data_for_df = segment_sections_into_subsections(updated_chapters_data)
data_for_df

,Chapter,Section_title,Section,Subsection
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...,
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...,
...,...,...,...,...
292,APPENDIX D,,"D.4 Techniques, Examples and Templates for Per...",D.4.1 Techniques for Perform Quantitative Risk...
293,APPENDIX D,,8.0 Primavera Pertmaster Figure D15. Example o...,7.3.8 from Oracle Hyperion Decisioneering
294,APPENDIX D,"D.5 Techniques, Examples, and Templates for Pl...",8 Plan Risk Responses develops the set of acti...,D.5.1 Techniques for Plan Risk Response Techni...
295,APPENDIX D,,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...


### Create the Subsection_title column

In [163]:
data_for_df['Subsection_title'] = None
for index, row in data_for_df.iterrows():
    subsection_title = ''
    for chapter, subsection_titles in chapter_subsections.items():
        for subtitle in subsection_titles:
            if subtitle.strip().lower() in row['Subsection'].strip().lower():
                subsection_title = subtitle.strip()
                data_for_df.at[index, 'Subsection'] = row['Subsection'].replace(subtitle, '').strip()
                break
        if section_title:
            break
    data_for_df.at[index, 'Subsection_title'] = subsection_title
data_for_df = data_for_df[['Chapter', 'Section_title', 'Section','Subsection_title','Subsection']]
data_for_df

,Chapter,Section_title,Section,Subsection_title,Subsection
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",,
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...,,
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...,,
...,...,...,...,...,...
292,APPENDIX D,,"D.4 Techniques, Examples and Templates for Per...",D.4.1 Techniques for Perform Quantitative Risk...,Examples of techniques for Perform Quantitativ...
293,APPENDIX D,,8.0 Primavera Pertmaster Figure D15. Example o...,,7.3.8 from Oracle Hyperion Decisioneering
294,APPENDIX D,"D.5 Techniques, Examples, and Templates for Pl...",8 Plan Risk Responses develops the set of acti...,D.5.1 Techniques for Plan Risk Response,Techniques for Plan Risk Responses are given i...
295,APPENDIX D,,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...


### Segment subsections into subsubsections

In [164]:
def segment_subsections_into_subsubsections(updated_clean_pmi):
    data_for_df = []
    for index, row in updated_clean_pmi.iterrows():
        chapter = row['Chapter']
        section_title = row['Section_title']
        section = row['Section']
        subsection_title = row['Subsection_title']
        subsection = row['Subsection']
        matches = list(re.finditer(r'\s([0-9]\.[0-9]\.[0-9]\.[0-9]|[A-Z]\.[0-9]\.[0-9]\.[0-9]+)\s', subsection))
        if matches:
            for match in matches:
                title_index = match.start()
                if title_index > 2 and (subsection[title_index - 2:title_index].lower() == 'and'
                                        or subsection[title_index - 2:title_index].lower() == 'in'
                                        or subsection[title_index - 2:title_index].lower() == 'through'
                                        or subsection[title_index - 2:title_index].lower() == 'sections'):
                    continue
                subsection_after_title = subsection[title_index:]
                updated_subsection = re.sub(r'\s([0-9]\.[0-9]\.[0-9]\.[0-9]|[A-Z]\.[0-9]\.[0-9]\.[0-9]+)\s', r' \\ \1 ', subsection_after_title)
                subsubsections = updated_subsection.split(' \\ ')
                subsubsections = [subsubsection.strip() for subsubsection in subsubsections if subsubsection.strip()]
                subsection = subsection.replace(subsection_after_title, '')
                if subsubsections:
                    for subsubsection in subsubsections:
                        data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section,'Subsection_title': subsection_title, 'Subsection': subsection, 'Subsubsection':subsubsection})
                else:
                    data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section,'Subsection_title': subsection_title, 'Subsection': subsection, 'Subsubsection':''})
        else:
            data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section,'Subsection_title': subsection_title, 'Subsection': subsection, 'Subsubsection':''})
    df = pd.DataFrame(data_for_df)
    return df

In [165]:
updated_data_for_df = segment_subsections_into_subsubsections(data_for_df)
updated_data_for_df.head(3)

,Chapter,Section_title,Section,Subsection_title,Subsection,Subsubsection
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,,,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,,,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",,,


In [166]:
def move_subtitles(df, subsubtitles):
    if 'Subsubsection_title' not in df.columns:
        df['Subsubsection_title'] = ''

    for subtitle in subsubtitles:
        df['Subsubsection_title'] = df.apply(
            lambda row: subtitle if subtitle in row['Subsubsection'] else row['Subsubsection_title'],
            axis=1
        )
        df['Subsubsection'] = df['Subsubsection'].apply(
            lambda text: text.replace(subtitle, '') if subtitle in text else text
        )

    return df
updated_data_for_df = move_subtitles(updated_data_for_df, subsubtitles)
updated_data_for_df

,Chapter,Section_title,Section,Subsection_title,Subsection,Subsubsection,Subsubsection_title
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,,,,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,,,,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",,,,
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...,,,,
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...,,,,
...,...,...,...,...,...,...,...
374,APPENDIX D,,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,,
375,APPENDIX D,,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,,
376,APPENDIX D,,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,,
377,APPENDIX D,,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,,


In [167]:
def process_dataframe(input_df):
    new_df = pd.DataFrame(columns=['Concept','Description'])
    input_df = input_df.drop_duplicates()
    chapter_to_title = {}
    section_title_seen = set()
    subsection_title_seen = set()
    for index, row in input_df.iterrows():
        chapter = row['Chapter']
        section = row['Section']
        subsection = row['Subsection']
        subsubsection = row['Subsubsection']
        section_title = row['Section_title']
        subsection_title = row['Subsection_title']
        subsubsection_title = row['Subsubsection']
        if chapter in chapter_to_title:

            if section_title in section_title_seen:

                if subsection_title in subsection_title_seen:
                    title = subsubsection_title
                    description=subsubsection

                elif subsection_title not in subsection_title_seen:
                    title = subsection_title
                    subsection_title_seen.add(subsection_title)
                    description=subsection

            elif section_title not in section_title_seen:
                title = section_title
                section_title_seen.add(section_title)
                description=section
        else:
            title = chapter
            chapter_to_title[chapter] = title
            description=section
        new_df = new_df.append({'Concept': title,'Description':description}, ignore_index=True)
    return new_df

In [168]:
df = process_dataframe(updated_data_for_df)
df

,Concept,Description
0,CHAPTER 1 INTRODUCTION,Project Management Institute PMI practice stan...
1,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...
2,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...
4,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...
...,...,...
163,Risks can and should be on the agenda at all ...,Risks can and should be on the agenda at all ...
164,The evolution of the variance values over tim...,The evolution of the variance values over tim...
165,"The formulae in Earned Value Analysis CV, SV,...","The formulae in Earned Value Analysis CV, SV,..."
166,,


### Extract concepts from the titles

In [169]:
def extract_concepts_from_titles(df):
    #stop_words = set(stopwords.words('english'))
    for index, row in df.iterrows():

        # Clean chapter titles
        chapter_header = r'CHAPTER (\d)+'
        row['Concept'] = re.sub(chapter_header, '', row['Concept'])

        # Clean section titles
        section_header = r'([1-9]\.[1-9]+|[A-Z]\.[1-9]+)'
        row['Concept'] = re.sub(section_header, '', row['Concept'])

        # Clean subsection titles
        subsection_header = r'([1-9]\.[1-9]\.[1-9]+|[A-Z]\.[1-9]\.[1-9]+)'
        row['Concept'] = re.sub(subsection_header, '', row['Concept'])

        # Clean subsubsection titles
        subsubsection_header = r'([1-9]\.[1-9]\.[1-9]\.[1-9]+|[A-Z]\.[1-9]\.[1-9]\.[1-9]+)'
        row['Concept'] = re.sub(subsubsection_header, '', row['Concept'])

        # Convert to lowercase
        #row['Concept'] = row['Concept'].lower()

        # Remove stopwords
        #words = row['Concept'].split()
        #filtered_words = [word for word in words if word.lower() not in stop_words]
        #row['Concept'] = ' '.join(filtered_words)

    return df

In [170]:
data=extract_concepts_from_titles(df)
data.head(20)

,Concept,Description
0,INTRODUCTION,Project Management Institute PMI practice stan...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...
4,Good Risk Management Practice,Project Risk Management is a valuable componen...
5,Critical Success Factors for Project Risk Man...,Specific criteria for success of each Project ...
6,Conclusion,The principles of Project Risk Management desc...
7,PRINCIPLES AND CONCEPTS,This chapter introduces the key ideas required...
8,Definition of Project Risk,The word risk is used in many ways in everyday...
9,Individual Risks and Overall Project Risk,It is useful to consider project risk at two l...


In [171]:
data['Concept'] = data['Concept'].str.replace(r'\.\d+', '', regex=True)

In [172]:
data.head(20)

,Concept,Description
0,INTRODUCTION,Project Management Institute PMI practice stan...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...
4,Good Risk Management Practice,Project Risk Management is a valuable componen...
5,Critical Success Factors for Project Risk Man...,Specific criteria for success of each Project ...
6,Conclusion,The principles of Project Risk Management desc...
7,PRINCIPLES AND CONCEPTS,This chapter introduces the key ideas required...
8,Definition of Project Risk,The word risk is used in many ways in everyday...
9,Individual Risks and Overall Project Risk,It is useful to consider project risk at two l...


In [173]:
data['Cleaned_Description'] = data['Description'].apply(clean_text)

In [174]:
data.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_1.csv', index=False)

# MERGING PMBOK + PMI

In [175]:
pmi = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_1.csv')
pmi.head()

,Concept,Description,Cleaned_Description
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ...",definition project risk management defined pmb...
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...,project risk management optional activity esse...
4,Good Risk Management Practice,Project Risk Management is a valuable componen...,project risk management valuable component pro...


In [179]:
pmbok = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_1.csv')
pmbok.drop('Unnamed: 0', axis=1, inplace=True)
pmbok.head()

,Concept,Description,Cleaned_Description
0,PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...
1,Plan Risk Management,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 plan risk managementplan risk management ...
2,Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...
3,Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques1...
4,Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...


In [180]:
dfs = [pmi, pmbok]
pmi_pmbok = pd.concat(dfs)
pmi_pmbok.sample(5)

,Concept,Description,Cleaned_Description
128,Risk identification interviews should include...,Risk identification interviews should include...,risk identification interviews include main st...
23,Implement Risk Responses Tools and Techniques,11.6.2.1 EXPERT JUDGMENT Described in Section ...,11.6.2.1 expert judgment described section 4.1...
22,Implement Risk Responses Inputs,11.6.1 IMPLEMENT RISK RESPONSES: INPUTS11.6.1....,11.6.1 implement risk responses: inputs11.6.1....
98,APPENDIX A,GUIDELINES FOR A PMI PRACTICE STANDARD,guidelines pmi practice standard
160,An analytical technique to determine the esse...,An analytical technique to determine the esse...,analytical technique determine essential featu...


In [182]:
pmi_pmbok.shape[0]

197

## NLP(Natural Language Processing)

### Sentence Tokenization and Removing Redanduncy
* We used sentence tokenization to split text into sentences because it helps in analyzing and processing text at a finer level, making it easier to extract meaning from individual statements.
* Lemmatization is used to reduce words to their base or dictionary form (lemma), which standardizes variations of a word and ensures that different forms of a word are treated as the same token. This is crucial for improving accuracy in text analysis and information retrieval. (this helped to remove redundancy because we have encountered issues like => "risks" != "risk")

In [183]:
def remove_repeated_and_plural_words(list_of_sentences):
    unique_sentences = []
    lemmatizer = WordNetLemmatizer()
    for sentence in list_of_sentences:
        words = sentence.split()
        unique_words = []
        for word in words:
            lemma_word = lemmatizer.lemmatize(word)
            if lemma_word not in unique_words:
                unique_words.append(lemma_word)
        unique_sentence = ' '.join(unique_words)
        unique_sentences.append(unique_sentence)
    return unique_sentences

In [186]:
#Sentence Tokenization using Regex because we have sections that are numbers concatenated by dots e.g: 11.2.3
pmi_pmbok['Sentences'] = pmi_pmbok['Cleaned_Description'].astype(str).apply(lambda x: re.split(r'(?<!\d)\.(?!\d)', x))
pmi_pmbok['Sentences'] = pmi_pmbok['Sentences'].apply(remove_repeated_and_plural_words)
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,Sentences
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...,[project management institute pmi practice sta...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...,[purpose practice standard project risk manage...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ...",definition project risk management defined pmb...,[definition project risk management defined pm...
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...,project risk management optional activity esse...,[project risk management optional activity ess...
4,Good Risk Management Practice,Project Risk Management is a valuable componen...,project risk management valuable component pro...,[project risk management valuable component en...


### POS TAGGING
POS (Part-of-Speech) tagging is used to label words in a text with their grammatical category (e.g., noun, verb, adjective) to understand the syntactic structure and relationships within sentences.
In our case we need the NOUNs to use as Concepts and VERBS as relationships (so we can create graphs)



In [187]:
def pos_tagging(list_of_sentences):
    pos_tagged_sentences = []
    for sentence in list_of_sentences:
        try:
            words = nltk.word_tokenize(sentence)
            pos_tags = nltk.pos_tag(words)
            pos_tagged_sentences.append(pos_tags)
        except Exception as e:
            print(f"Error processing sentence: {sentence}")
            continue
    return pos_tagged_sentences

In [188]:
pmi_pmbok['POS_Tags'] = pmi_pmbok['Sentences'].apply(pos_tagging)
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,Sentences,POS_Tags
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...,[project management institute pmi practice sta...,"[[(project, NN), (management, NN), (institute,..."
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...,[purpose practice standard project risk manage...,"[[(purpose, JJ), (practice, NN), (standard, NN..."
2,Project Risk Management Definition,"The definition of Project Risk Management, as ...",definition project risk management defined pmb...,[definition project risk management defined pm...,"[[(definition, NN), (project, NN), (risk, NN),..."
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...,project risk management optional activity esse...,[project risk management optional activity ess...,"[[(project, NN), (risk, NN), (management, NN),..."
4,Good Risk Management Practice,Project Risk Management is a valuable componen...,project risk management valuable component pro...,[project risk management valuable component en...,"[[(project, NN), (risk, NN), (management, NN),..."


### Extracting KeyWords Using KeyBert
Extracting keywords using KeyBert helps identify important concepts and themes in text, which is essential for creating a meaningful conceptual graph. KeyBert leverages contextual embeddings to find contextually relevant keywords, ensuring that the graph is based on the most significant terms. This process enhances the quality of the recommendation system, as it relies on a well-structured and semantically rich graph to make informed suggestions.

In [189]:
kw_model = KeyBERT(model='all-mpnet-base-v2')
def extract_concepts_keybert(text):
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1,3),
        stop_words='english',
        highlight=False,
        top_n=10
    )
    return [keyword[0] for keyword in keywords]

In [192]:
pmi_pmbok['KeyWords_KeyBert'] =  pmi_pmbok['Cleaned_Description'].astype(str).apply(extract_concepts_keybert)
pmi_pmbok[['Concept', 'KeyWords_KeyBert']]

,Concept,KeyWords_KeyBert
0,INTRODUCTION,"[pmi practice standards, pmi practice standard..."
1,Purpose of the Practice Standard for Project ...,"[project risk management, projects risk manage..."
2,Project Risk Management Definition,"[project risk management, definition project r..."
3,Role of Project Risk Management in Project Ma...,"[project risk management, project risk needs, ..."
4,Good Risk Management Practice,"[project risk management, management project r..."
...,...,...
24,Implement Risk Responses Outputs,"[risk responses project, project risk exposure..."
25,Monitor Risks,[risk reportprojectmanagementplanprojectdocume...
26,Monitor Risks Inputs,"[project risk exposure, project risks risk, pr..."
27,Monitor Risks Tools and Techniques,"[project risk management, project risks occur,..."


In [193]:
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,Sentences,POS_Tags,KeyWords_KeyBert
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...,[project management institute pmi practice sta...,"[[(project, NN), (management, NN), (institute,...","[pmi practice standards, pmi practice standard..."
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...,[purpose practice standard project risk manage...,"[[(purpose, JJ), (practice, NN), (standard, NN...","[project risk management, projects risk manage..."
2,Project Risk Management Definition,"The definition of Project Risk Management, as ...",definition project risk management defined pmb...,[definition project risk management defined pm...,"[[(definition, NN), (project, NN), (risk, NN),...","[project risk management, definition project r..."
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...,project risk management optional activity esse...,[project risk management optional activity ess...,"[[(project, NN), (risk, NN), (management, NN),...","[project risk management, project risk needs, ..."
4,Good Risk Management Practice,Project Risk Management is a valuable componen...,project risk management valuable component pro...,[project risk management valuable component en...,"[[(project, NN), (risk, NN), (management, NN),...","[project risk management, management project r..."


### Extracting Synonyms Using SBert
Extracting synonyms is valuable for enhancing natural language processing tasks and information retrieval. It broadens the search scope, improves recall, and ensures that a variety of terms can be used to find the same content. Synonyms help capture more comprehensive and relevant information when searching or analyzing text data.

In [210]:
def Synonym_extraction_SBert(df,title_column,keywords_column):
  All_Synonyms = []
  All_KeyWords = []
  for index, row in df.iterrows():
      sentences1 = [row[title_column]]
      sentences2 = row[keywords_column]
      embeddings1 = Sbert.encode(sentences1, convert_to_tensor=True)
      test = []
      synonym = []
      keywords = []
      for i in sentences2:
          embeddings2 = Sbert.encode(i, convert_to_tensor=True)
          cosine_scores = util.cos_sim(embeddings1, embeddings2)
          score = cosine_scores[0][0].item()
          if score >= 0.8:
              synonym.append(i)
          else:
              keywords.append(i)
          test.append(f'{sentences1[0]} <=> {i} Has the score: {score}')
      All_Synonyms.append(synonym)
      All_KeyWords.append(keywords)
  return All_Synonyms, All_KeyWords

In [212]:
pmi_pmbok['Concept'].fillna('', inplace=True)
pmi_pmbok['KeyWords_KeyBert'].fillna('', inplace=True)

In [213]:
pmi_pmbok['Synonyms_SBert'],pmi_pmbok['KeyWords_SBert'] = Synonym_extraction_SBert(pmi_pmbok,'Concept','KeyWords_KeyBert')
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,Sentences,POS_Tags,KeyWords_KeyBert,Synonyms_SBert,KeyWords_SBert
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...,[project management institute pmi practice sta...,"[[(project, NN), (management, NN), (institute,...","[pmi practice standards, pmi practice standard...",[],"[pmi practice standards, pmi practice standard..."
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...,[purpose practice standard project risk manage...,"[[(purpose, JJ), (practice, NN), (standard, NN...","[project risk management, projects risk manage...","[project risk management, projects risk manage...","[introduction project risk, standard project r..."
2,Project Risk Management Definition,"The definition of Project Risk Management, as ...",definition project risk management defined pmb...,[definition project risk management defined pm...,"[[(definition, NN), (project, NN), (risk, NN),...","[project risk management, definition project r...","[project risk management, definition project r...","[standard project risk, project risk, objectiv..."
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...,project risk management optional activity esse...,[project risk management optional activity ess...,"[[(project, NN), (risk, NN), (management, NN),...","[project risk management, project risk needs, ...","[project risk management, risk affects projects]","[project risk needs, risks project risk, proje..."
4,Good Risk Management Practice,Project Risk Management is a valuable componen...,project risk management valuable component pro...,[project risk management valuable component en...,"[[(project, NN), (risk, NN), (management, NN),...","[project risk management, management project r...",[risk management conducted],"[project risk management, management project r..."


In [214]:
pmi_pmbok.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_PMBOK.csv', index=False)

In [ ]:
#References(URL) #RelationExtraction(Rebel) #Definition(Bert Query)